# Generating geomedian composites SAR

* **Products used:** 
[s1_rtc](https://explorer.digitalearth.africa/s1_rtc)


## Background

Individual remote sensing images can be affected by noisy data, including clouds, cloud shadows, and haze. 
To produce cleaner images that can be compared more easily across time, we can create 'summary' images or 'composites' that combine multiple images into one image to reveal the median or 'typical' appearance of the landscape for a certain time period. 

One approach is to create a [geomedian](https://github.com/daleroberts/hdmedians). 
A `geomedian` is based on a high-dimensional statistic called the 'geometric median' [(Small 1990)](https://www.jstor.org/stable/1403809), which effectively trades a temporal stack of poor quality observations for a single high-quality pixel composite with reduced spatial noise [(Roberts et al. 2017)](https://ieeexplore.ieee.org/abstract/document/8004469). 
In contrast to a standard median, a geomedian maintains the relationship between spectral bands. 
This allows us to conduct further analysis on the composite images just as we would on the original satellite images (e.g by allowing the calculation of common band indices, like NDVI).

All the data of the selected timeframe has to be loaded to compute a composite, so geomedians can be computationally intensive to calculate, especially over large areas or long timescales. To assist with such analyses, DE Africa hosts a pre-calculated Sentinel-2 annual geomedian as part of the [Sentinel-2 GeoMAD](https://docs.digitalearthafrica.org/en/latest/data_specs/GeoMAD_specs.html) service. This reduces the time and resource needed to calculate the geomedian if you are conducting analysis over an annual timescale. Instructions on how to use the geomedian from the Sentinel-2 GeoMAD can be found in the [Datasets/GeoMAD.ipynb](../Datasets/GeoMAD.ipynb) notebook.

For analysis on other timescales, such as investigating change over seasons, it is not possible to use the annual geomedian product. In those cases, it can be useful to calculate geomedians for that specific time period.

## Description
In this notebook we will take of time series of noisy satellite images collected over six months and calculate a six-month geomedian composite which is largely free of clouds and other noisy data.

Geomedian computations are expensive in terms of memory, data bandwidth, and CPU usage. The ODC has some useful function, [int_geomedian](https://github.com/opendatacube/odc-tools/blob/f4f2126414ce807971f8be99dfc268669c36e7ad/libs/algo/odc/algo/_geomedian.py#L159) and [xr_geomedian](https://github.com/opendatacube/odc-tools/blob/f4f2126414ce807971f8be99dfc268669c36e7ad/libs/algo/odc/algo/_geomedian.py#L40) that allows [dask](https://docs.dask.org/en/latest/) to perform the computation in parallel across many threads to speed things up. In this notebook a `local dask cluster` is used, but the same approach should work using a larger, distributed dask cluster.

***


## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages

In [1]:
%matplotlib inline

import numpy as np
import datacube
from odc.algo import to_f32, xr_geomedian, int_geomedian
import warnings
warnings.filterwarnings('ignore')

from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import rgb
from deafrica_tools.dask import create_local_dask_cluster

import geopandas as gpd


### Set up a dask cluster

This will help keep our memory use down and conduct the analysis in parallel. If you'd like to view the `dask dashboard`, click on the hyperlink that prints below the cell. You can use the dashboard to monitor the progress of calculations.


In [2]:
create_local_dask_cluster()

Client Scheduler: tcp://127.0.0.1:39775 Dashboard: /user/timon.weitkamp@hotmail.com/proxy/46627/status,Cluster Workers: 1 Cores: 15 Memory: 104.37 GB


### Connect to the datacube

In [3]:
dc = datacube.Datacube(app='Generating_geomedian_composites_SAR')

## Load Sentinel-1 from the datacube

Here we are loading in a timeseries of cloud-masked Sentinel-2 satellite images through the datacube API using the [load_ard](../Frequently_used_code/Using_load_ard.ipynb) function. 
This will provide us with some data to work with. To limit computation and memory this example uses only three optical bands (red, green, blue).

In [4]:
# Set up centre of area of interest, and area to buffer coordinates by

vector_file = '../data/buffered.geojson'  #contains the study area coordinates
gdf = gpd.read_file(vector_file)

area = "Chokwe"
gdf_area = gdf[(gdf['area'] == area)]
gdf_area

lat = gdf_area.lat #+ 0.09 #TOP
lon = gdf_area.lon
buffer_lon = 0.18 #buffer
buffer_lat = 0.18

#below are the calculations of the various composite lenghts

# 1 x 12 months
#date = ('2019-10','2020-09'); file_name = area + "_12mo_20192020_SAR"

# 2 x 6 months
#date = ('2019-10','2020-03'); file_name = area + "_6mo_2019Q4_2020Q1_SAR"
#date = ('2020-04','2020-09'); file_name = area + "_6mo_2020Q2_2020Q3_SAR"

date = ('2021-10','2022-03'); file_name = area + "_6mo_2021Q4_2022Q1_SAR"
#date = ('2022-04','2022-09'); file_name = area + "_6mo_2022Q2_2022Q3_SAR"

# 4 x 3 months
#date = ('2021-10','2021-12'); file_name = area + "_3mo_2021_Q4_SAR"
#date = ('2022-01','2022-03'); file_name = area + "_3mo_2022_Q1_SAR"
#date = ('2022-04','2022-06'); file_name = area + "_3mo_2022_Q2_SAR"
#date = ('2022-06','2022-09'); file_name = area + "_3mo_2022_Q3_SAR"

#date = ('2019-10','2019-12'); file_name = area + "_3mo_2019_Q4_SAR"
#date = ('2020-01','2020-03'); file_name = area + "_3mo_2020_Q1_SAR"
#date = ('2020-04','2020-06'); file_name = area + "_3mo_2020_Q2_SAR"
#date = ('2020-06','2020-09'); file_name = area + "_3mo_2020_Q3_SAR"

# 6 x 2 months
#date = ('2019-10','2019-11'); file_name = area + "_2mo_2019_10_11_SAR"
#date = ('2019-12','2020-01'); file_name = area + "_2mo_2020_12_01_SAR"
#date = ('2020-02','2020-03'); file_name = area + "_2mo_2020_02_03_SAR"
#date = ('2020-04','2020-05'); file_name = area + "_2mo_2020_04_05_SAR"
#date = ('2020-06','2020-07'); file_name = area + "_2mo_2020_06_07_SAR"
#date = ('2020-08','2020-09'); file_name = area + "_2mo_2020_08_09_SAR"

# Create a reusable query
query = {
    'x': (lon-buffer_lon, lon+buffer_lon),
    'y': (lat+buffer_lat, lat-buffer_lat),
    'time': date,
    'measurements': ['vv','vh'],
    'resolution': (-10, 10),
    'group_by': 'solar_day',
    'output_crs': 'EPSG:6933'
}

Compared to the typical use of `load_ard` which by default returns data with floating point numbers containing `NaN` (i.e. `float32`), in this example we will set the `dtype` to `'native'`. 
This will keep our data in its original integer data type (i.e. `Int16`), with nodata values marked with `-999`.
Doing this will halve the amount of memory our data takes up, which can be extremely valuable when conducting large-scale analyses.

In [5]:
ds = load_ard(dc=dc, 
              products=['s1_rtc'],
              dask_chunks={'x':3000, 'y':3000},
              dtype = 'native',
              **query)

# Print output data
print(ds)

Using pixel quality parameters for Sentinel 1
Finding datasets
    s1_rtc
Applying pixel quality/cloud mask
Returning 57 time steps as a dask array
<xarray.Dataset>
Dimensions:      (time: 57, y: 4189, x: 3475)
Coordinates:
  * time         (time) datetime64[ns] 2021-10-04T03:18:18.332440 ... 2022-03...
  * y            (y) float64 -3.016e+06 -3.016e+06 ... -3.058e+06 -3.058e+06
  * x            (x) float64 3.167e+06 3.167e+06 ... 3.201e+06 3.201e+06
    spatial_ref  int32 6933
Data variables:
    vv           (time, y, x) float32 dask.array<chunksize=(1, 3000, 3000), meta=np.ndarray>
    vh           (time, y, x) float32 dask.array<chunksize=(1, 3000, 3000), meta=np.ndarray>
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref


## Plot timesteps in true colour

To visualise the data, use the pre-loaded `rgb` utility function to plot a true colour image for a series of timesteps. 
Black areas indicate where clouds or other invalid pixels in the image have been set to `-999` to indicate no data.

The code below will plot three timesteps of the time series we just loaded.

> **Note:** This step can be quite slow because the dask arrays being plotted must be computed first.

In [6]:
#import matplotlib.pyplot as plt


# Plot the first VH and VV observation for the year 2020.
#fig, ax = plt.subplots(1, 2, figsize=(14, 6), sharey=True)
#ds.vh.isel(time=0).plot.imshow(cmap="Greys_r", robust=True, ax=ax[0])
#ds.vv.isel(time=0).plot.imshow(cmap="Greys_r", robust=True, ax=ax[1])
#ax[0].set_title("VH")
#ax[1].set_title("VV");



## Generate a geomedian

As you can see above, most satellite images will have at least some areas masked out due to clouds or other interference between the satellite and the ground. 
Running the `int_geomedian` function will generate a geomedian composite by combining all the observations in our `xarray.Dataset` into a single, complete (or near complete) image representing the geometric median of the time period.

> **Note:** Because our data was lazily loaded with `dask`, the geomedian algorithm itself will not be triggered until we call the `.compute()` method in the next step.

In [7]:
geomedian = int_geomedian(ds)

### Run the computation

The `.compute()` method will trigger the computation of the geomedian algorithm above.
This will take about a few minutes to run; view the `dask dashboard` to check the progress.

In [8]:
%%time
geomedian = geomedian.compute()

CPU times: user 22.4 s, sys: 3.28 s, total: 25.6 s
Wall time: 24min 19s


If we print our result, you will see that the `time` dimension has now been removed and we are left with a single image that represents the geometric median of all the satellite images in our initial time series:

In [9]:
geomedian

<xarray.Dataset>
Dimensions:  (y: 4189, x: 3475)
Coordinates:
  * y        (y) float64 -3.016e+06 -3.016e+06 ... -3.058e+06 -3.058e+06
  * x        (x) float64 3.167e+06 3.167e+06 3.167e+06 ... 3.201e+06 3.201e+06
Data variables:
    vv       (y, x) float32 0.08435 0.08435 0.1112 ... 0.1061 0.1061 0.09667
    vh       (y, x) float32 0.02113 0.02113 0.02397 ... 0.02485 0.02485 0.02396
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref

### Plot the geomedian composite

Plotting the result, we can see that the geomedian image is much more complete than any of the individual images. 
We can also use this data in downstream analysis as the relationships between the spectral bands are maintained by the geometric median statistic.


In [11]:
from deafrica_tools.bandindices import dualpol_indices

# Calculate NDVI using `calculate indices`
geomedian = dualpol_indices(geomedian, index=['RVI'])
geomedian

<xarray.Dataset>
Dimensions:  (y: 4189, x: 3475)
Coordinates:
  * y        (y) float64 -3.016e+06 -3.016e+06 ... -3.058e+06 -3.058e+06
  * x        (x) float64 3.167e+06 3.167e+06 3.167e+06 ... 3.201e+06 3.201e+06
Data variables:
    vv       (y, x) float32 0.08435 0.08435 0.1112 ... 0.1061 0.1061 0.09667
    vh       (y, x) float32 0.02113 0.02113 0.02397 ... 0.02485 0.02485 0.02396
    RVI      (y, x) float32 0.8013 0.8013 0.7095 0.7095 ... 0.759 0.759 0.7944
Attributes:
    crs:           EPSG:6933
    grid_mapping:  spatial_ref

In [12]:
from datacube.utils.cog import write_cog

write_cog(geomedian.to_array(),
          fname= "MAD/S1/"+file_name+".tif", #"MAD/Nabusenga2019sMADQ3.tif",
          overwrite=True)

PosixPath('MAD/S1/Chokwe_6mo_2021Q4_2022Q1.tif')

In [13]:
from IPython.lib.display import Audio
import numpy as np

framerate = 3000
play_time_seconds = 1

t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
audio_data = np.sin(3*np.pi*300*t) + np.sin(3*np.pi*240*t)
Audio(audio_data, rate=framerate, autoplay=True)

In [14]:
#geomedian.RVI.plot(col = 'time')

In [15]:
#geomedian.VDDPI.plot()

In [16]:
#geomedian.ratio.plot()

In [17]:
#geomedian.purity.plot()

In [18]:
#geomedian.entropy.plot()

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:**

In [19]:
print(datacube.__version__)

1.8.7


**Last Tested:**

In [20]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2022-09-22'